## Imports

In [1]:
from about_data import * 
from clean import *

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Data

In [13]:
messy_df = pd.read_csv(data_file, sep='\t', low_memory=False)

In [12]:
universal_features, nonuniversal_features = get_universals(txt_path, csv_root_path)

,CASEID,V07312,V07319,V07320,V09154,V08825,V07477,V07292,V07297,V08249,...,V04611,NCSRWTSH,NCSRWTLG,NCNSWTSH,NCNSWTLG,NCNLWTSH,NCNLWTLG,NSNLWT,CPESWTSH,CPESWTLG
0,1,,,,1,2,0,0,2,2,...,,,,,,11627.4082,14040.66,15968.9517,11627.3949,14040.6279
1,2,,,,1,2,0,0,1,1,...,,,,,,10090.8728,12185.2188,13858.6912,10090.8613,12185.1909
2,3,,,,3,2,0,0,1,1,...,,,,,,10090.8728,12185.2188,13858.6912,10090.8613,12185.1909
3,4,,,,1,9,0,0,2,2,...,,,,,,5939.8128,6525.4281,7919.7406,5939.8017,6525.416
4,5,,,,1,6,0,0,1,1,...,,,,,,4572.4542,5023.2596,6096.5981,4572.4457,5023.2502


In [21]:
clean_df = pd.DataFrame()

In [23]:
for item in good_features: 
    clean_df[item] = messy_df[item]

In [45]:
for feature in good_features:
    print(feature, pd.unique(clean_df[feature]))

V01626 [' ' '5' '1' '7']
V01627 [' ' '1' '5' '7']
V01628 [' ' '5' '1' '7']
V01629 [' ' '5' '1' '7' '-8']
V01630 [' ' '1' '5' '7' '-8']
V01631 [' ' '5' '1' '7']
V01632 [' ' '1' '5' '-8' '7']
V01633 [' ' '1' '5' '-8' '7']
V01634 [' ' '1' '5' '7']
V01635 [' ' '1' '5' '7' '-8']
V01636 [' ' '5' '1' '7']
V01638 [' ' '1' '5' '-8' '-9']
V01639 [' ' '-8' '1' '5']
V01640 [' ' '20' '18' '16' '13' '9' '30' '43' '45' '19' '14' '50' '48' '8' '31'
 '34' '15' '11' '12' '6' '35' '5' '33' '28' '26' '10' '55' '29' '22' '23'
 '7' '17' '1' '27' '52' '59' '42' '36' '46' '37' '49' '21' '44' '39' '40'
 '60' '51' '4' '25' '41' '38' '86' '32' '24' '53' '70' '80' '47' '56' '58'
 '73' '63' '57' '3' '67' '65' '-8']
V01642 [' ' '11' '15' '18' '9' '13' '6' '53' '26' '-8' '28' '22' '20' '7' '10'
 '47' '59' '8' '3' '4' '5' '14' '17' '30' '19' '31' '33' '16' '48' '23'
 '40' '12' '21' '50' '24' '42' '25' '32' '35' '72' '39' '44' '54' '70' '27'
 '34' '61' '45' '38' '60' '52']
V01643 [' ' '5' '1' '-8']
V01644 [' ' '1' '5'

V04322 ['1' ' ' '2' '-8' '-9']
V04341 ['2' '1' '3' '4' '-8' ' ' '-9']
V04342 ['2' '1' '4' '3' '-8' ' ' '-9']
V04343 ['3' '2' '4' '1' '-8' ' ' '-9']
V04351 ['5' '1' ' ' '-9']
V04352 ['0' '-9' '3' '5' '1' '20' '10' '2' '-8' ' ' '4' '100' '7' '12' '18' '6'
 '15' '30' '25' '999' '35' '200']
V04353 [' ' '5' '1' '-8']
V04354 [' ' '22' '42' '16' '18' '19' '28' '24' '17' '29' '26' '21' '37' '33' '15'
 '31' '32' '-8' '43' '23' '27' '45' '39' '25' '41' '5' '55' '44' '70' '35'
 '30' '11' '20' '40' '13' '14' '51' '58' '-9' '34' '46' '12' '6' '1' '48'
 '49' '36' '7' '50']
V04355 [' ' '14' '5' '0' '15' '30' '19' '-8' '12' '11' '55' '22' '17' '16' '36'
 '29' '28' '18' '20' '25' '27' '23' '24' '21' '26' '-9' '43' '32' '13' '39'
 '38' '45' '33' '47' '37']
V04356 [' ' '5' '1']
V04357 ['0' '1' '10' '5' '2' '4' '-9' '3' '6' '-8' '7' '20' '100' '9' '8' '50' ' '
 '12' '15' '250' '995' '1000' '30' '40' '500' '999' '11' '25' '200']
V06120 [' ' '5' '1' '-8' '-9']
V06121 [' ' '5' '1' '-9' '-8']
V06122 [' ' '5' 

V09396 ['17' '16' '14' ' ' '10' '11' '12' '13' '6' '15' '8' '5' '4' '9' '-8' '7'
 '-9']
V09398 ['1' '2' ' ' '6' '4' '3' '97' '5' '-8' '-9']
V09399 [' ' '1' '6' '4' '2' '97' '5' '3']
V09400 [' ' '2' '4' '3' '5']
V09402 ['1' ' ' '2' '3' '-9' '-8']
V09403 [' ' '1' '0' '2' '3' '-9']
V09404 ['28' '27' '24' '25' '-9' ' ' '26' '16' '19' '23' '22' '32' '21' '29' '30'
 '17' '20' '18' '31' '15' '34' '-8' '35' '33' '42' '14' '39' '40' '38' '36'
 '37' '41']
V09405 [' ' '37' '3' '6' '7' '2' '-8' '26' '27' '4' '5' '1' '30' '-9' '10' '21'
 '22' '15' '9' '8' '18' '113' '12' '11' '40' '16' '20' '14' '17' '19' '24'
 '13' '42' '28' '44' '25' '47' '49' '31' '32' '33' '0' '50' '56' '45' '34'
 '36' '39']
V09406 [' ' '4' '1' '3' '2' '-8' '-9']
V09409 ['0' '2' '6' '3' '1' '4' '5' '-8' '-9' ' ']
V09413 ['0' '2' '3' '1' '5' '4' '6' '-8' '-9' ' ']
V09414 ['14' '8' '9' '10' '13' '7' '12' '6' '1' '5' '4' '2' '3' '-9' '11' '-8' ' ']
V00829 [' ' '1' '5' '-8' '-9']
V00830 [' ' '1' '5' '-8']
V00831 [' ' '1' '5' '-8' '

V04691 [' ' '1' '5']
V04692 [' ' '1' '5' '-8']
V04694 [' ' '1' '5' '-8']
V04695 [' ' '20' '43' '18' '52' '40' '22' '28' '10' '16' '26' '15' '13' '23' '21'
 '42' '25' '9' '12' '8' '31' '14' '19' '36' '27' '30' '45' '35' '38' '34'
 '32' '7' '33' '41' '55' '17' '60' '46' '44' '11' '29' '59' '6' '24' '51'
 '49' '77' '47' '53' '39' '37' '5' '1' '4']
V04697 [' ' '-8' '10' '27' '37' '32' '20' '35' '24' '21' '16' '13' '38' '30' '40'
 '19' '50' '14' '15' '12' '60' '28' '9' '7' '26' '17' '34' '23' '25' '6'
 '18' '29' '8' '5' '11' '42' '59' '48' '47' '52' '81']
V04698 [' ' '1' '5' '-8']
V04699 [' ' '20' '10' '2' '1' '0' '4' '3' '-8' '8' '9' '12' '5' '17' '15' '7' '25'
 '6' '26' '30' '60' '16' '13' '-9' '19' '35' '18' '14' '55' '23' '32' '11'
 '24' '48' '120' '33' '39']
V04701 [' ' '3' '4' '1' '2' '-8' '-9']
V04702 [' ' '30' '25' '33' '17' '12' '47' '29' '13' '27' '36' '37' '21' '18' '45'
 '10' '20' '48' '43' '26' '16' '50' '51' '41' '75' '38' '53' '64' '52' '31'
 '39' '22' '23' '49' '40' '-8' '-9

V01181 [' ' '5' '1' '-9']
V01183 [' ' '3' '4' '1' '5' '2']
V01184 [' ' '2' '1' '3' '4']
V01185 [' ' '4' '3' '2' '1' '-8']
V01186 [' ' '4' '2' '3' '1' '-8']
V01187 [' ' '1' '2' '3' '4']
V01189 [' ' '5' '1' '-8']
V01190 [' ' '5' '1']
V01191 [' ' '1' '5' '-8']
V01192 [' ' '39' '13' '17' '21' '30' '14' '5' '27' '24' '45' '18' '11' '36' '26'
 '31' '50' '38' '25' '19' '15' '41' '20' '23' '37' '28' '16' '6' '7' '32'
 '35' '47' '10' '40' '9' '22' '8' '48' '29' '49' '70' '42' '12' '54' '58'
 '43' '34' '60' '33' '51' '65' '56' '59' '46' '78' '62' '52' '66' '44' '53']
V01194 [' ' '29' '14' '16' '13' '-8' '12' '26' '23' '27' '8' '10' '15' '30' '9'
 '19' '43' '35' '18' '45' '32' '22' '17' '6' '25' '37' '66' '7' '36' '21'
 '39' '31' '-9' '60' '5' '11' '28' '76' '40' '20' '33' '50' '34' '42' '38']
V01195 [' ' '5' '-8' '1']
V01196 [' ' '5' '1' '-8']
V01199 [' ' '3' '1' '2' '10' '9' '4' '5' '12' '-8' '6' '11' '42' '15' '7' '8' '14'
 '45' '27' '13' '20' '30' '38' '16' '-9' '0' '18' '35' '17' '23' '25']


...

V02892 [' ' '8' '9' '14' '11' '7' '13']
V02869 [' ' '9' '13' '8']
V02870 [' ' '10' '9']
V02871 [' ' '11' '10']
V02872 [' ' '14' '11']
V02873 [' ' '16' '12']
V02894 [' ' '1' '2' '3' '-8' '4' '-9']
V02898 [' ' '1' '2' '-8' '-9']
V02908 [' ' '1' '2' '3' '4' '5' '6' '-8']
V02909 [' ' '28' '5' '25' '0' '-7' '4' '1' '-8' '30' '3' '2' '7' '10' '20' '21'
 '8' '15' '14' '6' '27' '18' '12' '-9']
V02919 [' ' '5' '1' '-8' '-9']
V02930 [' ' '5' '1' '7' '-8']
V02931 [' ' '5' '7' '1' '-8']
V02932 [' ' '1' '2' '-8']
V02936 [' ' '9' '6' '4' '1' '3' '7' '2' '5' '-8']
V02941 [' ' '7' '5' '2' '3' '6' '9' '8']
V02944 [' ' '7' '3' '9' '5' '4' '8']
V02958 [' ' '15' '14' '5' '1' '6' '7' '8' '10' '3' '4' '2']
V02965 [' ' '14' '15' '11' '6' '5' '3' '10']
V02974 [' ' '8' '1' '16' '2' '11' '18' '7' '-8' '5' '9' '3' '10' '17' '-9' '15'
 '4']
V02984 [' ' '8' '5' '2' '3' '7' '4' '9' '17' '1' '18' '16' '15']
V02987 [' ' '7' '4' '2' '6' '18' '13' '10' '1' '16' '3' '12' '8' '5' '17' '9']
V02988 [' ' '9' '7' '5' '18' '3

V03653 [' ' '5' '1' '2' '3' '4' '-8' '-9']
V03655 [' ' '2' '4' '5' '3' '1' '-8' '-9']
V03656 [' ' '4' '5' '3' '2' '1' '-8' '-9']
V03659 [' ' '1']
V03661 [' ' '1' '5' '-8']
V03663 [' ' '-8' '51' '-9' '32' '10' '11' '43' '7' '37']
V03664 [' ' '1' '5' '-9' '-8']
V03667 [' ' '1']
V03678 [' ' '1']
V03689 [' ' '1']
V03691 [' ' '1']
V03692 [' ' '1']
V03693 [' ' '1']
V03694 [' ' '1']
V03695 [' ' '1']
V03696 [' ' '1']
V03668 [' ' '1']
V03669 [' ' '1']
V03670 [' ' '1']
V03671 [' ' '1']
V03672 [' ' '1']
V03673 [' ' '1']
V03674 [' ' '1']
V03675 [' ' '1']
V03699 [' ' '1']
V03903 [' ' '5' '1' '-8' '-9']
V03905 [' ' '5' '1' '-8' '-9']
V03906 [' ' '1' '5' '-8' '-9']
V03907 [' ' '5' '1' '-8' '-9']
V03908 [' ' '1' '5' '-8' '-9']
V03909 [' ' '5' '1' '-8' '-9']
V03910 [' ' '1' '5' '-9' '-8']
V03911 [' ' '1' '5' '-9' '-8']
V03912 [' ' '5' '1' '-9' '-8']
V03913 [' ' '1' '5' '-9' '-8']
V03914 [' ' '1' '5' '-9']
V03915 [' ' '1' '5' '-9' '-8']
V03916 [' ' '5' '1' '-8' '-9']
V03920 [' ' '0' '1' '18' '2' '5' '10

V02334 [' ' '8']
V02336 [' ' '5' '1' '-8' '-9']
V02337 [' ' '5' '1' '-8' '-9']
V02338 [' ' '1' '5' '-8' '-9']
V02339 [' ' '5' '1' '-8' '-9']
V02340 [' ' '1' '5' '-8' '-9']
V02341 [' ' '5' '1' '-8' '-9']
V02342 [' ' '1' '5' '-8' '-9']
V02343 [' ' '5' '1' '-8' '-9']
V02344 [' ' '5' '1' '-8' '-9']
V02345 [' ' '1' '5' '-8' '-9']
V02346 [' ' '5' '1' '-8' '-9']
V02347 [' ' '5' '1' '-8' '-9']
V02348 [' ' '1' '5' '-8' '-9']
V02349 [' ' '1' '5' '-8' '-9']
V02350 [' ' '5' '1' '-8' '-9']
V02354 [' ' '1' '5' '-8' '-9']
V02355 [' ' '5' '1' '-9' '-8']
V02356 [' ' '10' '4' '7' '1' '5' '6' '8' '-8' '2' '15' '13' '3' '16' '40' '12'
 '34' '0' '20' '30' '9' '18' '24' '11' '50' '-9' '60' '14' '64' '52' '25'
 '23' '49' '90' '28' '22' '29']
V02357 [' ' '3' '1' '4' '2' '-8' '-9']
V02358 [' ' '1' '2' '4' '3' '-8' '-9']
V02359 [' ' '5' '1' '-8' '-9']
V02361 [' ' '5' '1' '-8']
V02362 [' ' '1' '5' '-8']
V02363 [' ' '1' '5' '-8']
V02364 [' ' '1' '5' '-8']
V02365 [' ' '1' '5' '-8']
V02366 [' ' '5' '1' '-8']
V02367

V02036 [' ' '5' '1']
V02037 [' ' '20' '13' '26' '38' '12' '19' '24' '18' '17' '59' '-8' '23' '41' '46'
 '22' '30' '33' '35' '15' '16' '14' '37' '36' '21' '43' '44' '42' '32' '31'
 '29' '25' '45' '50' '5' '40' '34' '39' '55' '28']
V02041 [' ' '3' '1' '2' '-8']
V06390 ['5' '1' ' ' '-8' '-9']
V06403 ['5' '1' ' ' '-9' '-8']
V01743 [' ' '3' '30' '6' '4' '1' '7' '13' '9' '20' '12' '25' '16' '2' '10' '5' '8'
 '27' '11' '32' '24' '14' '29' '19' '21' '31' '28' '-9' '-8' '15' '26' '23'
 '22' '17' '18']
V01744 [' ' '2' '19' '8' '3' '10' '25' '6' '13' '4' '17' '11' '16' '7' '20' '28'
 '5' '23' '1' '30' '21' '24' '9' '29' '27' '12' '15' '22' '31' '14' '18'
 '26' '32']
V01745 [' ' '4' '20' '9' '28' '7' '24' '31' '5' '16' '3' '6' '8' '15' '19' '10'
 '25' '22' '11' '13' '1' '21' '14' '30' '12' '32' '17' '29' '2' '26' '18'
 '27' '23']
V01746 [' ' '8' '5' '16' '26' '4' '10' '6' '24' '11' '25' '3' '23' '29' '12' '9'
 '20' '22' '14' '18' '28' '27' '19' '7' '31' '17' '15' '1' '21' '13' '30'
 '2' '32']
V017

V08516 ['5' '1' ' ']
V07348 [' ' '18' '15' '34' '22' '14' '19' '25' '13' '17' '16' '21']
V07354 [' ' '16' '21' '23' '15' '20' '26' '14' '31' '22' '18' '34' '17' '44']
V08321 ['5' '1' ' ']
V08517 ['5' '1' ' ']
V07532 ['5' ' ']
V07530 ['5' '1' ' ']
V07834 ['5' '1' ' ']
V07346 [' ' '18' '15' '33' '22' '14' '19' '25' '13' '17' '16' '21' '12']
V07352 [' ' '16' '21' '23' '15' '20' '26' '14' '31' '22' '18' '34' '35' '17' '30'
 '44']
V07546 ['5' '1' ' ']
V07545 ['5' '1' ' ']
V07839 ['5' '1' ' ']
V07401 [' ' '20' '43' '18' '52' '40' '22' '28' '10' '16' '19' '26' '15' '13' '23'
 '21' '42' '25' '27' '9' '12' '30' '31' '37' '32' '14' '35' '24' '36' '45'
 '38' '7' '33' '41' '55' '17' '60' '44' '11' '29' '59' '50' '8' '34' '51'
 '6' '49' '77' '47' '53' '5' '39' '48' '4' '81']
V07402 [' ' '45' '30' '43' '20' '18' '54' '40' '22' '26' '28' '31' '25' '59' '33'
 '17' '21' '12' '47' '10' '15' '16' '29' '27' '34' '13' '32' '36' '37' '14'
 '41' '35' '24' '38' '48' '19' '49' '50' '51' '42' '75' '52' '60' '53

In [ ]:
range_to_5 = []
for feature in good_features:
    if 
    print(feature, pd.unique(clean_df[feature]))

In [68]:
x = [' ' '1' '5' '7' '-8']

In [56]:
ints = list(map(int, x))

ValueError: invalid literal for int() with base 10: ' 157-8'

In [72]:
for item in x: 
    print(item)

 157-8


## Cleaning Functions Test Area

### Local file paths 

In [2]:
data_file = '/Users/Winnifred/Desktop/Capstone/ICPSR_20240_RAWDATA/DS0001/20240-0001-Data.tsv'
txt_path = '/Users/Winnifred/Desktop/Capstone/diagnosis_capstone/data/feature_group_file_names.txt'
csv_root_path = '/Users/Winnifred/Desktop/Capstone/diagnosis_capstone/data/feature_name_data/'

### Test Area

In [7]:
universal_features, nonuniversal_features = get_universals(txt_path, csv_root_path)

In [19]:
good_features = list(universal_features.keys())

In [20]:
good_features

['V01626',
 'V01627',
 'V01628',
 'V01629',
 'V01630',
 'V01631',
 'V01632',
 'V01633',
 'V01634',
 'V01635',
 'V01636',
 'V01638',
 'V01639',
 'V01640',
 'V01642',
 'V01643',
 'V01644',
 'V01646',
 'V01647',
 'V01648',
 'V01649',
 'V01650',
 'V01651',
 'V01652',
 'V01653',
 'V01656',
 'V01657',
 'V01658',
 'V01659',
 'V01660',
 'V01661',
 'V01662',
 'V01663',
 'V01664',
 'V01666',
 'V01667',
 'V01668',
 'V01669',
 'V01670',
 'V01671',
 'V01672',
 'V01673',
 'V01674',
 'V01675',
 'V01676',
 'V01677',
 'V01678',
 'V01679',
 'V01680',
 'V01681',
 'V01682',
 'V01683',
 'V01684',
 'V01685',
 'V01686',
 'V01687',
 'V01688',
 'V01689',
 'V01690',
 'V01691',
 'V01692',
 'V01695',
 'V01697',
 'V01709',
 'V01730',
 'V01731',
 'V01732',
 'V01733',
 'V01734',
 'V01735',
 'V01736',
 'V01737',
 'V05448',
 'V05500',
 'V05434',
 'V05436',
 'V04044',
 'V04045',
 'V04046',
 'V04047',
 'V04048',
 'V04049',
 'V04050',
 'V04051',
 'V04052',
 'V04053',
 'V04055',
 'V04056',
 'V04057',
 'V04059',
 'V04060',

## IMPORTANT TO DO ITEMS
- Look at distribution of features between the categories (i.e. what proportion of each group made it through the universals test)